## Import all Libraries

In [1]:
# AWS Rekognition to get bbox
import numpy as np
import boto3
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont
from matplotlib import pyplot as plt
from utils.rekognition import determine_color, draw_animal_count
import time
import math
import os
import glob
import io
import json
FNT = ImageFont.truetype('/usr/share/fonts/default/Type1/n019004l.pfb', 25)

## Get Bounding Boxes from Video Frames

In [2]:
#draw response
def draw_response(image, response, animal_target):
    # original image size
    draw = ImageDraw.Draw(image)

    for customLabel in response['CustomLabels']:

        if 'Geometry' in customLabel:
            box = customLabel['Geometry']['BoundingBox']
            left = box['Left']
            top = box['Top']
            width = box['Width']
            height = box['Height']
            #print(customLabel['Name'])
            label = customLabel['Name']
        
            text = label
            text_width, text_height = FNT.getsize(label)
            color = determine_color(label, True)

            button_width = int(text_width + 20)
            button_height = int(text_height + 15)
            button_size = (button_width, button_height)
            button_img = Image.new('RGB', button_size, color)
            button_draw = ImageDraw.Draw(button_img)
            button_draw.text((10, 10), text, fill ='#000000', font=FNT)
            image.paste(button_img, (int(left), int(top)))  

            points = (
                (left, top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top))
            
            thickness = 5
            
            if label == 'cow':
                thickness = 7
                
            draw.line(points, fill=color, width=thickness)
    
    img = np.asarray(image)[:,:,::-1].copy()
    return img

In [5]:
def get_bbox_json(src_file, output_file, projectVersionArn, min_confidence=75):
    start = time.time()
    rekognition = boto3.client('rekognition')
    count = 0
    # get total frames
    frame_total = 0
    for infile in os.listdir(src_file):
        if infile.endswith(".jpg"):
            frame_total += 1
    print('Total frames:', frame_total)
    frame_res = [{} for _ in range(frame_total)]
    # process each image file
    for infile in os.listdir(src_file):
        if infile.endswith(".jpg"):
            file, ext = os.path.splitext(infile)
            frameId = int(file)
            #print('FrameId', frameId)
            with Image.open(src_file+file+ext) as img:
                #print(img.size)
                img_byte_arr = io.BytesIO()
                img.save(img_byte_arr, format='PNG')
                img_byte_arr = img_byte_arr.getvalue()
                # get labels
                response = rekognition.detect_custom_labels(
                    Image={
                        'Bytes': img_byte_arr,
                    },
                    ProjectVersionArn=projectVersionArn
                )
                imgWidth, imgHeight = img.size
                # scale bbox coordiantion to image size 
                for idx, customLabel in enumerate(response['CustomLabels']):
                    if 'Geometry' in customLabel:
                        box = customLabel['Geometry']['BoundingBox']
                        box['Left'] = imgWidth * box['Left']
                        box['Top'] = imgHeight * box['Top']
                        box['Width'] = imgWidth * box['Width']
                        box['Height'] = imgHeight * box['Height']
#                 if frameId == 200:
#                     inferred_frame = draw_response(img, response, animal_target='cow')
#                     plt.imshow(inferred_frame)
#                     plt.title("Frame {}".format(int(frameId)))
#                     plt.show()
                frame_res[frameId] = {'CustomLabels': response['CustomLabels']}
                    
            count+=1
            if count % 50 == 0:
                print("Finish Processing {} frame".format(count))
                lap = time.time()
                print('lap time: ', lap - start)
                
    #print(frame_res[617])
    #the json file where the output must be stored
    with open(output_file, 'w+') as out_file:
        json.dump({'Frames':frame_res}, out_file)
        out_file.close()
    end = time.time()
    print('total time lapse', end - start)

In [7]:
six_class_arn = 'arn:aws:rekognition:us-east-1:617989056061:project/cow-detector/version/cow-detector.2021-05-27T13.18.14/1622146694516'
# 'Maternity Pen 1_00_20210420044929', 'Maternity Pen 1_00_20210420044420', 'Maternity Pen 1_00_20210420044503'
#, 'cattle_multi_1', 'cattle_single_1' 
video_name_list = ['IMG_4195']
for video_name in video_name_list:
    src_path = 'frame_img/'+video_name+'/'
    output_json_path = 'json_data/'+video_name+'_new_bbox.json'
    print(src_path)
    print(output_json_path)
    get_bbox_json(src_path, output_json_path, six_class_arn)
print('finished estimating bbox.')

frame_img/IMG_4195/
json_data/IMG_4195_new_bbox.json
Total frames: 1070
Finish Processing 50 frame
lap time:  172.79816961288452
Finish Processing 100 frame
lap time:  344.93002009391785
Finish Processing 150 frame
lap time:  520.2203125953674
Finish Processing 200 frame
lap time:  694.6408205032349
Finish Processing 250 frame
lap time:  869.5612018108368
Finish Processing 300 frame
lap time:  1074.9598941802979
Finish Processing 350 frame
lap time:  1283.820217370987
Finish Processing 400 frame
lap time:  1494.0098254680634
Finish Processing 450 frame
lap time:  1703.5177686214447
Finish Processing 500 frame
lap time:  1911.6847484111786
Finish Processing 550 frame
lap time:  2131.7788105010986
Finish Processing 600 frame
lap time:  2343.2823219299316
Finish Processing 650 frame
lap time:  2556.795689344406
Finish Processing 700 frame
lap time:  2762.72598862648
Finish Processing 750 frame
lap time:  2976.6706562042236
Finish Processing 800 frame
lap time:  3181.534832239151
Finish Pr